In [8]:
import numpy as np
import pandas as pd
import plotly.express as px

In [9]:
f = open("data/lichess_db_standard_rated_2018-01.pgn")

In [10]:
def get_next_analysed_game():
    # Пока не найдем игру с анализом
    while True:
        # Записываем строки игры
        buffer = []
        while True:
            line = f.readline()

            # Пустые строки не нужны
            if line == "\n":
                continue

            buffer.append(line)

            # Ходы идут последней строкой в игре
            # и всегда начинаются с "1."
            if line.startswith("1."):
                break
        
        if ("%eval" in line):
            return buffer
        
def params_to_dict(str_list):
    """
    Принимает строки в виде
    [key "value"]
    И переводит их в словарь
    https://en.wikipedia.org/wiki/Portable_Game_Notation
    """
    return {
        a: b.strip('"') 
        for a, b in [
            i.strip("\n").strip("[]").split(" ", 1) 
            for i in str_list
        ]
    }

def moves_to_dict(moves):
    s = moves.split(" ")
    s = s[:-1]
    
    # Если последний ход - мат, для него нет оценки позиции (игра окончена)
    # Поэтому слайсы сдвигаются, время не в той колонке
    # Чтобы исправить, чуть подправим в конце
    if len(s) % 8 != 0:
        s.append(s[-2])
        s[-3] = "Game Over"
    
    return {
        "WhiteMove": s[1::16],
        "WhiteEval": [i.strip("[]") for i in s[4::16]],
        "WhiteTime": [i.strip("[]") for i in s[6::16]],
        
        "BlackMove": s[9::16],
        "BlackEval": [i.strip("[]") for i in s[12::16]],
        "BlackTime": [i.strip("[]") for i in s[14::16]],
    }

def get_and_parse_next_analysed_game():
    game = get_next_analysed_game()
    return {
        **params_to_dict(game[:-1]),
        **moves_to_dict(game[-1])
    }

In [11]:
def get_moves_df(game):
    df = pd.DataFrame.from_dict({
        
        "WhiteMove": game["WhiteMove"],
        "WhiteEval": game["WhiteEval"],
        "WhiteTime": game["WhiteTime"],
        
        "BlackMove": game["BlackMove"],
        "BlackEval": game["BlackEval"],
        "BlackTime": game["BlackTime"],
    
    }, orient="index").transpose()
    
    df["MoveNumber"] = df.index + 1
    
    return df

In [12]:
def get_two_dfs():
    a = get_and_parse_next_analysed_game()
    m = get_moves_df(a)
    return a, m

In [14]:
a, m = get_two_dfs()

In [16]:
m

,WhiteMove,WhiteEval,WhiteTime,BlackMove,BlackEval,BlackTime,MoveNumber
0,e4,0.09,0:01:00,c5,0.17,0:01:00,1
1,Nf3,0.03,0:00:59,e6,0.16,0:00:59,2
2,b4,-0.03,0:00:55,cxb4,-0.12,0:00:58,3
3,a3,-0.05,0:00:54,Nf6,-0.06,0:00:56,4
4,e5,-0.05,0:00:53,Nd5,-0.06,0:00:55,5
5,axb4,-0.15,0:00:52,Bxb4,-0.12,0:00:53,6
6,c3,-0.19,0:00:52,Ba5?!,0.45,0:00:52,7
7,d4?,-2.4,0:00:51,Nc6?,0.12,0:00:51,8
8,Bd2,0.26,0:00:47,O-O,0.35,0:00:48,9
9,Bc4?!,-0.6,0:00:43,d6,-0.45,0:00:47,10


In [ ]:
## Notes

In [6]:
# Дальше 100 хода анализа нет, их можно отфильтровать

# for i in range(10_000):
#     game = get_next_analysed_game()  
#     table = get_moves_df(moves_to_dict(game[-1]))
#     if len(table) >= 100:
#         print(game[-1])

In [7]:
# Может быть неопределенный исход
# [Result "*"]
# Их лучше исключить из анализа

In [ ]:
# for i in range(10_000):
#     g = get_and_parse_next_analysed_game()
#     if int(g["WhiteElo"]) < 800:
#         print(g["Site"])